In [1]:
import pandas as pd

In [2]:
Data = pd.read_csv(r"C:\Users\Carlo\Desktop\Todo el proyecto\Data Posición\Medios.csv")
pd.set_option('display.max_columns', None)

In [3]:
import pandas as pd
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler
import plotly.graph_objects as go

Columna_habilidades = Data.columns.drop(["Name", "League"])

features = Data[Columna_habilidades]

# Crea un objeto StandardScaler
scaler = StandardScaler()

# Ajusta el escalador a tus datos y luego transforma tus datos
features_scaled = scaler.fit_transform(features)

# Inicializa el modelo NearestNeighbors
model = NearestNeighbors(n_neighbors=6, algorithm = "auto")  # Buscamos 6 vecinos porque uno de ellos será el jugador mismo

# Ajusta el modelo a tus datos
model.fit(features_scaled)

caracteristicas_grafico = ['Goals', 'Accurate final third passes','Interceptions', 'Big chances created','Assists','Was fouled', 'Fouls']

def buscar_jugadores_similares(nombre, caracteristicas_grafico):
    jugador = Data[Data['Name'] == nombre]
    if len(jugador) == 0:
        return 'Jugador no encontrado'
    else:
        jugador_scaled = scaler.transform(jugador[features.columns])
        distancias, indices = model.kneighbors(jugador_scaled)
        jugadores_similares = Data.iloc[indices[0][1:]]  # Excluimos el primer resultado porque será el jugador mismo

        # Escala todas las características de los jugadores similares
        jugadores_similares_scaled = scaler.transform(jugadores_similares[features.columns])

        # Crea un gráfico de radar para cada jugador similar
        fig = go.Figure()
        for i, row in enumerate(jugadores_similares_scaled):
            fig.add_trace(go.Scatterpolar(
                r=row[features.columns.isin(caracteristicas_grafico)],
                theta=caracteristicas_grafico,
                fill='toself',
                name=jugadores_similares.iloc[i]['Name']
            ))
        fig.update_layout(
            autosize=False,
            width=800,
            height=800,
            polar=dict(
                radialaxis=dict(
                    visible=True,
                    range=[-2, 10]  # Ajusta este rango según tus datos
                )),
            showlegend=True
        )
        fig.show()

        return jugadores_similares


In [4]:
buscar_jugadores_similares('Eder González', caracteristicas_grafico)

,Name,Goals,Successful dribbles %,Total shots,Set piece conversion %,Tackles,Interceptions,Dribbled past,Big chances created,Assists,Total passes,Accurate passes %,Accurate final third passes,Accurate long balls %,Aerial duels won %,Total duels won %,Was fouled,Fouls,Appearances,Age Range,Position,League,Level
5055,Matheus Bueno,0.03,71.43,0.66,0.0,1.45,1.17,0.76,0.03,0.00,47.86,84.58,7.62,56.71,47.50,57.09,2.21,1.52,29,2,3,Brasil Serie B,3
483,Iker Undabarrena,0.00,55.56,0.15,0.0,1.97,1.61,0.85,0.06,0.06,48.52,85.88,8.03,61.34,60.00,57.61,1.58,1.39,33,2,3,España La Liga 2,2
3117,Radosław Murawski,0.03,70.00,0.45,0.0,2.00,1.32,0.74,0.10,0.06,46.48,84.39,9.10,64.54,50.00,55.38,1.23,1.65,31,2,3,Polonia Ekstraklasa,3
2968,Frank Ysique,0.00,60.00,0.30,0.0,1.60,1.43,0.57,0.03,0.07,45.57,88.30,8.43,72.73,38.46,52.88,1.33,1.50,30,2,3,Peru Liga 1,3
2135,Kei Koizumi,0.03,60.98,0.53,0.0,2.20,1.40,0.90,0.03,0.07,56.83,90.62,11.50,55.56,33.33,54.04,1.67,1.67,30,2,3,Japon J1 League,4
